In [1]:
"""
Δημιουργία του πίνακα chartevet με το είδος των εξετάσων διαχωρισμένες σε 707 στήλες.
"""

'\nΔημιουργία του πίνακα chartevet με το είδος των εξετάσων διαχωρισμένες σε 707 στήλες.\n'

In [2]:
#Εισαγωγή βιβλιοθήκης Panda
import pandas as pd

In [3]:
#Δημιουργώ τις στήλες από το αρχείο που περιλαμβάνει τις μοναδικές εξετάσεις
df_columns = pd.read_csv('D:\MSc Aegean\Διατριβή\MIMIC_IV\mimic-iv-1.0\exports\o9_cerebral_infarction_chartevents_counts_with_labels.csv',
                         usecols = ['itemid'], index_col=False)

#Ορίζω το header ως index έτσι ώστε όταν αλλάξω την κατεύθυνση του csv να μπουν τα στοιχεία στην κεφαλίδα.
df_columns = df_columns.set_index('itemid')

# Μεταφέρω τις γραμμές σε στήλες
df_columns = df_columns.transpose()

In [4]:
df_rows = pd.read_csv('D:\MSc Aegean\Διατριβή\MIMIC_IV\mimic-iv-1.0\exports\o9_charttime_8_hour_mean_item_merged.csv')

C:\Users\Dimopoulos\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [5]:
# Μετατροπή της στήλης charttime από object σε datetime για να μπορέσει να γίνει ο παρακάτω διαχωρισμός σε δυο στήλες
df_rows['charttime']= pd.to_datetime(df_rows['charttime'])

In [6]:
#Διαχωρισμός σε δυο καινούργιες στήλες της ημερομηνίας και ώρας
df_rows['date'] = pd.to_datetime(df_rows['charttime']).dt.date
df_rows['time'] = pd.to_datetime(df_rows['charttime']).dt.time

In [7]:
#Αλλαγή της σειράς εμφάνισης των στοιχείων του dataframe, για να είναι ευανάγνωστο κατά τους ελέγχους μου,
#δεν είναι απαραίτητο για την εξέλιξη του κώδικα
df_rows = df_rows[['subject_id', 'hadm_id', 'charttime', 'date', 'time', 'itemid', 'Mean_Chart']]

In [8]:
df_rows.to_csv('D:\MSc Aegean\Διατριβή\MIMIC_IV\mimic-iv-1.0\exports\o9_cerebral_infarction_chartevents_grouped_8hours_date_time.csv', index=False)

In [9]:
# Εδώ παίρνουμε τις τιμές της στήλης time και βλέπουμε πως γκρουπάρονται οι ώρες.
# Οπ΄ότε έχουμε τρία οχτάωρα 00:00, 08:00 και 16:00
list(set(df_rows['time']))

[datetime.time(0, 0), datetime.time(8, 0), datetime.time(16, 0)]

In [10]:
# Απλά προβάλω τις στήλες των καταγραφών
df_columns

itemid,220045,220046,220047,220050,220051,220052,220056,220058,220059,220060,...,228396,228869,228870,228872,228873,228874,228875,228876,229319,229355


In [11]:
"""
Δημιουργώ ένα μοναδικό αναγνωριστικό με την ένωση των
στοιχείων από στήλες subject, hadm_id, date και time.
Ανάμεσα στα στοιχεία προβάλω ένα κενό το οποίο θα μου
φανεί χρήσιμο αργότερα για τον διαχωρισμό τους ξανά σε
διαφορετικές στήλες.
"""
df_rows["concat"] = (df_rows["subject_id"].astype(str)
                     +" "
                     +df_rows["hadm_id"].astype(str)
                     +" "
                     +df_rows["date"].astype(str)
                     +" "+
                     df_rows["time"].astype(str))

In [12]:
# Ελέγχω αν υπάρχει κενό κελί στην στήλη concat.
df_rows['concat'].isnull().values.any()

False

In [13]:
# Αλλάζω την σειρά των στηλών του dataframe
df_rows = df_rows[['concat', 'subject_id', 'hadm_id', 'charttime', 'date', 'time', 'itemid', 'Mean_Chart']]

In [14]:
# Από το dr_rows κρατάω τις μοναδικές εγγραφές
df_copy_rows = pd.DataFrame(df_rows.concat.unique())

# Δημιουργώ καινούργιο πίνακα ενσωματώνοντας στον πίνακα df columns τις στήλες με τις κενές για την ώρα εξετάσεις
chart_tbl = pd.concat([df_copy_rows, df_columns], axis=1)

# Μετονομάζω την στήλη 0 σε concat για να είναι ίδιες στους δυο πίνακες
chart_tbl = chart_tbl.rename({0:'concat'}, axis=1)

In [15]:
#Διαχωρισμός του concat σε τέσσερις στήλες
chart_tbl[['subject_id','hadm_id', 'date', 'time']] = chart_tbl['concat'].str.split(expand=True)

In [16]:
#Διαγράφω την στήλη concat γιατί θα πρέπει να την ξαναδημιουργήσω
#χωρίς κενά ανάμεσα στα ενοποιημένα στοιχεία, οπότε θα την έχω διπλή.
chart_tbl = chart_tbl.drop('concat', 1)

In [17]:
chart_tbl

,220045,220046,220047,220050,220051,220052,220056,220058,220059,220060,...,228873,228874,228875,228876,229319,229355,subject_id,hadm_id,date,time
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,10004733,27411876,2174-12-04,08:00:00
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,10004733,27411876,2174-12-04,16:00:00
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,10004733,27411876,2174-12-05,00:00:00
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,10004733,27411876,2174-12-05,08:00:00
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,10004733,27411876,2174-12-05,16:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65184,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,10546885,25213587,2174-09-29,08:00:00
65185,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,11114858,24037757,2128-08-12,08:00:00
65186,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,19623640,29398204,2168-04-01,16:00:00
65187,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,11974151,22782098,2152-07-02,16:00:00


In [18]:
# Εξαγωγή του άδειου πίνακα για να τον γεμίσουμε στο επόμενο στάδιο
chart_tbl.to_csv('D:\MSc Aegean\Διατριβή\MIMIC_IV\mimic-iv-1.0\exports\o9_empty_chart_tbl.csv', index=False)